Kategorik degiskenler ile otomatik olarak mucadele edebilen hizli ve basarili bir GBM turu

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import scale, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR


from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
df = pd.read_csv('Hitters.csv')
df = df.dropna()
dms= pd.get_dummies(df[['League','Division','NewLeague']]) #kategorik degiskenleri dummy degiskenlere donusturduk
y=df["Salary"]
X_ = df.drop(['Salary','League','Division','NewLeague'],axis=1).astype('float64')# bu 4 sutun kaldirir kalan sutunlari da float64 veri turune cevirir
X=pd.concat([X_,dms[['League_N','Division_W','NewLeague_N']]],axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [3]:
!pip install catboost

In [7]:
from catboost import CatBoostRegressor

In [9]:
catb_model = CatBoostRegressor().fit(X_train,y_train)

Learning rate set to 0.031674
0:	learn: 437.6430699	total: 56.7ms	remaining: 56.7s
1:	learn: 431.3923642	total: 57.8ms	remaining: 28.8s
2:	learn: 424.8820360	total: 59ms	remaining: 19.6s
3:	learn: 418.2514904	total: 60.8ms	remaining: 15.1s
4:	learn: 412.6394021	total: 61.9ms	remaining: 12.3s
5:	learn: 406.6247020	total: 62.7ms	remaining: 10.4s
6:	learn: 400.5321206	total: 63.4ms	remaining: 9s
7:	learn: 394.6683437	total: 64.3ms	remaining: 7.97s
8:	learn: 388.2496484	total: 65.8ms	remaining: 7.25s
9:	learn: 382.9448842	total: 67.4ms	remaining: 6.67s
10:	learn: 377.2600080	total: 68.9ms	remaining: 6.19s
11:	learn: 372.4829606	total: 71.5ms	remaining: 5.88s
12:	learn: 366.6823437	total: 72.9ms	remaining: 5.53s
13:	learn: 362.6076230	total: 73.7ms	remaining: 5.19s
14:	learn: 358.0107745	total: 74.5ms	remaining: 4.89s
15:	learn: 353.2802665	total: 75.2ms	remaining: 4.62s
16:	learn: 348.5646265	total: 76ms	remaining: 4.4s
17:	learn: 343.6407912	total: 76.9ms	remaining: 4.2s
18:	learn: 339.23

In [11]:
y_pred = catb_model.predict(X_test)

In [13]:
np.sqrt(mean_squared_error(y_test,y_pred))

351.194631344607

Model Tuning

In [16]:
catb_params = {"iterations" : [200,500,100],
               "learning_rate" : [0.01,0.1],
               "depth":[3,6,8]}

In [18]:
catb_model = CatBoostRegressor()

In [22]:
catb_cv_model = GridSearchCV(catb_model , catb_params , cv=5,n_jobs = -1 , verbose =2).fit(X_train,y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
0:	learn: 423.4375355	total: 57.5ms	remaining: 11.4s
1:	learn: 421.1765508	total: 57.8ms	remaining: 5.72s
2:	learn: 419.0857364	total: 59ms	remaining: 3.87s
3:	learn: 416.9452654	total: 59.1ms	remaining: 2.9s
4:	learn: 414.7312054	total: 59.2ms	remaining: 2.31s
5:	learn: 412.7163500	total: 59.4ms	remaining: 1.92s
6:	learn: 410.8901481	total: 59.5ms	remaining: 1.64s
7:	learn: 408.9530169	total: 59.6ms	remaining: 1.43s
8:	learn: 407.1127322	total: 59.7ms	remaining: 1.27s
9:	learn: 404.9830351	total: 59.9ms	remaining: 1.14s
10:	learn: 402.8219949	total: 60ms	remaining: 1.03s
11:	learn: 400.8572723	total: 60.2ms	remaining: 944ms
12:	learn: 399.1287133	total: 60.4ms	remaining: 869ms
13:	learn: 397.2954867	total: 60.7ms	remaining: 806ms
14:	learn: 395.2920342	total: 60.9ms	remaining: 751ms
15:	learn: 393.5502831	total: 61.1ms	remaining: 703ms
16:	learn: 391.6542300	total: 61.3ms	remaining: 660ms
17:	learn: 389.8509633	total: 62.9ms

In [24]:
catb_cv_model.best_params_

{'depth': 3, 'iterations': 200, 'learning_rate': 0.1}

In [26]:
catb_tuned = CatBoostRegressor(depth=3,iterations = 200,learning_rate =0.1).fit(X_train,y_train)

0:	learn: 425.7900818	total: 1.37ms	remaining: 272ms
1:	learn: 404.8723520	total: 2.04ms	remaining: 202ms
2:	learn: 387.4057666	total: 2.37ms	remaining: 156ms
3:	learn: 372.2801584	total: 2.71ms	remaining: 133ms
4:	learn: 358.9204229	total: 3.14ms	remaining: 122ms
5:	learn: 347.0083933	total: 3.44ms	remaining: 111ms
6:	learn: 336.0130818	total: 3.76ms	remaining: 104ms
7:	learn: 324.3923300	total: 4.06ms	remaining: 97.5ms
8:	learn: 314.8690957	total: 4.38ms	remaining: 92.9ms
9:	learn: 308.5075563	total: 4.67ms	remaining: 88.8ms
10:	learn: 298.8587285	total: 4.95ms	remaining: 85ms
11:	learn: 294.7655438	total: 5.46ms	remaining: 85.6ms
12:	learn: 288.0697862	total: 5.77ms	remaining: 83ms
13:	learn: 282.6697154	total: 6.03ms	remaining: 80.1ms
14:	learn: 277.6121667	total: 6.3ms	remaining: 77.8ms
15:	learn: 273.4383979	total: 6.61ms	remaining: 76ms
16:	learn: 269.1556201	total: 6.88ms	remaining: 74.1ms
17:	learn: 264.8098704	total: 7.21ms	remaining: 72.9ms
18:	learn: 261.6700768	total: 7.49

In [28]:
y_pred = catb_tuned.predict(X_test)

In [30]:
np.sqrt(mean_squared_error(y_test,y_pred))

344.3125832615482